In [17]:
import pandas as pd
import numpy as np
import sklearn
df=pd.read_csv('homedata.csv')
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df, df['size_cat']):
    train_set = df.iloc[train_index]
    test_set = df.iloc[test_index]
train_set.drop('size_cat', axis=1, inplace=True)
test_set.drop('size_cat', axis=1, inplace=True)
housing = train_set.drop('price', axis=1)
housing_labels = train_set['price'].copy()
housing_num = housing[['rooms', 'size', 'level', 'max_levels']]

from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, size_ix = 2, 3
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def fit(self, X):
        return self
    def transform(self, X):
        size_per_room = X[:, size_ix] / X[:, rooms_ix]
        return np.c_[X, size_per_room]

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_pipline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
])


C:\Users\Asrorbek\AppData\Local\Temp\ipykernel_9296\3486069929.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set.drop('size_cat', axis=1, inplace=True)
C:\Users\Asrorbek\AppData\Local\Temp\ipykernel_9296\3486069929.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set.drop('size_cat', axis=1, inplace=True)


In [18]:
num_pipline.fit_transform(housing_num)

array([[ 0.34724433, -0.54338637, -0.31492312, -0.77535811, -0.5234883 ],
       [-0.57951842, -0.29950045,  0.56624035, -0.39157678, -0.7321414 ],
       [ 0.34724433,  0.32376358,  0.12565861, -0.77535811, -0.7321414 ],
       ...,
       [ 0.34724433,  0.1882714 , -1.19608658, -0.77535811,  1.14573645],
       [-0.57951842, -0.62468168, -1.19608658, -0.77535811,  1.14573645],
       [-0.57951842, -0.57048481,  0.12565861, -0.39157678, -0.57565158]])

In [19]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ['district']

full_pipeline = ColumnTransformer([
    ('num', num_pipline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

In [20]:
housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared

array([[ 0.34724433, -0.54338637, -0.31492312, ...,  0.        ,
         0.        ,  0.        ],
       [-0.57951842, -0.29950045,  0.56624035, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.34724433,  0.32376358,  0.12565861, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.34724433,  0.1882714 , -1.19608658, ...,  0.        ,
         0.        ,  0.        ],
       [-0.57951842, -0.62468168, -1.19608658, ...,  0.        ,
         0.        ,  0.        ],
       [-0.57951842, -0.57048481,  0.12565861, ...,  1.        ,
         0.        ,  0.        ]])

In [21]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression()
LR_model.fit(housing_prepared, housing_labels)

LinearRegression()

In [22]:
test_set

,Unnamed: 0,location,district,rooms,size,level,max_levels,price
6471,6563,"город Ташкент, Чиланзарский район, Шарк",Чиланзарский,2,55.0,4,4,36500
3660,3720,"город Ташкент, Юнусабадский район, Юнусобод 19",Юнусабадский,2,60.0,1,5,42000
383,385,"город Ташкент, Яккасарайский район, Глинки",Яккасарайский,2,52.0,1,3,55000
7018,7129,"город Ташкент, Учтепинский район, Гулистон",Учтепинский,2,40.0,2,4,36000
3039,3077,"город Ташкент, Яшнободский район, жилой компле...",Яшнободский,5,120.0,7,9,88000
...,...,...,...,...,...,...,...,...
5027,5107,"город Ташкент, Яшнободский район, Фергана Йули",Яшнободский,3,72.0,5,9,36500
6315,6405,"город Ташкент, Чиланзарский район, Алмазар",Чиланзарский,2,59.0,2,9,51000
3796,3856,"город Ташкент, Мирзо-Улугбекский район, Карасу...",Мирзо-Улугбекский,2,90.0,2,9,51000
2049,2077,"город Ташкент, Сергелийский район, СЕРГЕЛИ-7",Сергелийский,5,120.0,9,9,59000


In [23]:
X_test = test_set.drop('price', axis=1)
y_test = test_set['price'].copy()

In [24]:
X_test_prepared = full_pipeline.transform(X_test)


In [25]:
y_predicted = LR_model.predict(X_test_prepared)

In [26]:
y_predicted

array([38729.99809134, 52449.18270146, 48133.75393573, ...,
       82558.7660072 , 68762.48993983, 73280.73853956])

In [27]:
y_test

6471    36500
3660    42000
383     55000
7018    36000
3039    88000
        ...  
5027    36500
6315    51000
3796    51000
2049    59000
5602    77000
Name: price, Length: 1490, dtype: int64

In [28]:
# Baholash. Root mean square error (RMSE)
from sklearn.metrics import mean_squared_error
lin_mse = mean_squared_error(y_test, y_predicted)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

32057.274606296287


In [29]:
# DECISION TREE

from sklearn.tree import DecisionTreeRegressor
Tree_model = DecisionTreeRegressor()
Tree_model.fit(housing_prepared, housing_labels)

y_predicted = Tree_model.predict(X_test_prepared)
lin_mse = mean_squared_error(y_test, y_predicted)
lin_rmse = np.sqrt(lin_mse)    
print(lin_rmse)

35775.465215681


In [30]:
# RANDOMFOREST

from sklearn.ensemble import RandomForestRegressor
RF_model = RandomForestRegressor()
RF_model.fit(housing_prepared, housing_labels)

y_predicted = RF_model.predict(X_test_prepared)
lin_mse = mean_squared_error(y_test, y_predicted)
lin_rmse = np.sqrt(lin_mse)    
print(lin_rmse)

30845.1279147272
